# 02 - Actualización de Datos de Mercado

Este notebook actualiza los precios históricos de todos los activos en cartera.

**Proceso:**
1. Lee todos los históricos de cuentas corrientes
2. Extrae lista única de tickers
3. Descarga precios faltantes desde Yahoo Finance
4. Actualiza `data/02_processed/prices/master_prices.csv`

In [ ]:
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta

# Agregar src al path
sys.path.append(str(Path.cwd().parent / 'src'))

from core.pricing_engine import PricingEngine
from utils.file_io import load_yaml_config

In [ ]:
# Configuración de rutas
BASE_PATH = Path.cwd().parent
CONFIG_PATH = BASE_PATH / 'config'
DATA_PATH = BASE_PATH / 'data'
PROCESSED_PATH = DATA_PATH / '02_processed'
ACCOUNTS_PATH = PROCESSED_PATH / 'accounts'

# Inicializar Pricing Engine
pricing_engine = PricingEngine(PROCESSED_PATH, CONFIG_PATH)

In [ ]:
# Leer todos los históricos de cuentas corrientes
account_files = list(ACCOUNTS_PATH.glob('*_cc_historica.csv'))

print(f"Archivos de cuentas encontrados: {len(account_files)}")

all_tickers = set()

for file in account_files:
    df = pd.read_csv(file)
    if 'ticker' in df.columns:
        tickers = df['ticker'].dropna().unique()
        all_tickers.update(tickers)

all_tickers = sorted(list(all_tickers))
print(f"\nTickers únicos encontrados: {len(all_tickers)}")
print(f"Primeros 10: {all_tickers[:10]}")

In [ ]:
# Determinar rango de fechas a actualizar
master_prices = pricing_engine.get_master_prices()

if master_prices is not None and not master_prices.empty:
    last_date = master_prices['date'].max()
    start_date = (last_date + timedelta(days=1)).strftime('%Y-%m-%d')
    print(f"Última fecha en master_prices: {last_date}")
    print(f"Actualizando desde: {start_date}")
else:
    # Si no hay datos, descargar últimos 2 años
    start_date = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')
    print(f"No hay datos previos, descargando desde: {start_date}")

end_date = datetime.now().strftime('%Y-%m-%d')
print(f"Hasta: {end_date}")

In [ ]:
# Descargar precios
print("\nDescargando precios...\n")
new_prices = pricing_engine.download_prices(all_tickers, start_date, end_date)

if not new_prices.empty:
    print(f"\nPrecios descargados: {len(new_prices)} registros")
    print(f"Tickers con datos: {new_prices['ticker'].nunique()}")
    
    # Actualizar master
    pricing_engine.update_master_prices(new_prices)
else:
    print("\n⚠️  No se descargaron nuevos precios")

In [ ]:
# Verificar cobertura
master = pricing_engine.get_master_prices()

if master is not None:
    print("\nEstadísticas del master_prices:")
    print(f"  Total registros: {len(master):,}")
    print(f"  Tickers únicos: {master['ticker'].nunique()}")
    print(f"  Rango fechas: {master['date'].min()} a {master['date'].max()}")
    
    # Tickers sin datos
    tickers_en_master = set(master['ticker'].unique())
    tickers_faltantes = set(all_tickers) - tickers_en_master
    
    if tickers_faltantes:
        print(f"\n⚠️  Tickers sin datos ({len(tickers_faltantes)}):")
        for ticker in sorted(tickers_faltantes):
            print(f"    - {ticker}")

In [ ]:
print("\n✓ Actualización de precios completada")